In [ ]:
import time
import torch

from diffusers import StableDiffusionPipeline, DDIMScheduler

In [ ]:
def save_and_display(image, pred_path):
    timestr = time.strftime("%Y%m%d-%H%M%S")
    image.save(pred_path + "/" + timestr + ".png")
    display(image)

In [ ]:
model_path =  "/home/ubuntu/model"
predict_path = "/home/ubuntu/predict"
!mkdir -p "$predict_path"

token_name = "aabbccddeeffgg"
class_str = "person"
feature_str = ", detailed face"

num_pred_steps = 75
guide = 7.5
ddim = True

if ddim:
    ddim_scheduler = DDIMScheduler(
            beta_start=0.00085,
            beta_end=0.012,
            beta_schedule="scaled_linear",
            clip_sample=False,
            set_alpha_to_one=False,
        )

    pipe = StableDiffusionPipeline.from_pretrained(
        model_path, 
        torch_dtype=torch.float16,
        scheduler=ddim_scheduler,
        safety_checker=None).to("cuda")
else:
    pipe = StableDiffusionPipeline.from_pretrained(
        model_path, 
        torch_dtype=torch.float16,
        safety_checker=None).to("cuda")      

In [ ]:
prompt = "photo of __token__ __class__ __feature__, astronaut, helmet, oil painting, greg rutkowski"
fill_placeholders = lambda x: x.replace("__token__", token_name).replace("__class__", class_str).replace("__feature__", feature_str)
prompt = fill_placeholders(prompt)

In [ ]:
generator = torch.Generator(device="cuda")
seed = generator.seed()
generator = generator.manual_seed(seed)

width = 512
height = 512
latents = torch.randn(
    (1, pipe.unet.in_channels, height // 8, width // 8),
    generator = generator,
    device = "cuda"
).half()

image = pipe(prompt, latents = latents, num_inference_steps=num_pred_steps, guidance_scale=guide).images[0]
print(prompt)
save_and_display(image, predict_path)

In [ ]:
# less steps
image = pipe(prompt, latents = latents, num_inference_steps=25, guidance_scale=guide).images[0]
print(prompt)
save_and_display(image, predict_path)

In [ ]:
# more steps
image = pipe(prompt, latents = latents, num_inference_steps=125, guidance_scale=guide).images[0]
print(prompt)
save_and_display(image, predict_path)

In [ ]:
# even more steps
image = pipe(prompt, latents = latents, num_inference_steps=200, guidance_scale=guide).images[0]
print(prompt)
save_and_display(image, predict_path)

In [ ]:
# weaker guide
image = pipe(prompt, latents = latents, num_inference_steps=num_pred_steps, guidance_scale=2.5).images[0]
print(prompt)
save_and_display(image, predict_path)

In [ ]:
# stronger guide
image = pipe(prompt, latents = latents, num_inference_steps=num_pred_steps, guidance_scale=15).images[0]
print(prompt)
save_and_display(image, predict_path)

In [ ]:
# stronger guide + more steps
image = pipe(prompt, latents = latents, num_inference_steps=200, guidance_scale=15).images[0]
print(prompt)
save_and_display(image, predict_path)

In [ ]:
# No ddim
pipe_2 = StableDiffusionPipeline.from_pretrained(
    model_path, 
    torch_dtype=torch.float16,
    safety_checker=None).to("cuda")
image = pipe_2(prompt, latents = latents, num_inference_steps=200, guidance_scale=15).images[0]
print(prompt)
save_and_display(image, predict_path)